<a href="https://colab.research.google.com/github/ascle/colab_tcc2/blob/dev/6_melhores_features_rf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importando bibliotecas

In [30]:
import pandas as pd
import seaborn as sns
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import metrics

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from functools import reduce

time: 6.26 ms (started: 2023-09-18 00:44:31 +00:00)


### Outras configurações<hr style='border: 2px solid red;'>

In [31]:
pd.set_option('display.max_columns', None)
SEED = 7
np.random.seed(SEED)

!pip install ipython-autotime
pd.set_option('display.max_columns', None)
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 5.13 s (started: 2023-09-18 00:44:35 +00:00)


### Conectando no ambiente que será utilizado

In [3]:
# Importando do Drive
from google.colab import drive
from os import chdir

drive.mount('/content/drive', force_remount=True)
chdir('/content/drive/MyDrive/UFS/tcc-2-ascle-ufs/bases/linkage_marco/')

Mounted at /content/drive
time: 24 s (started: 2023-09-17 23:20:54 +00:00)


### Leitura dos dados <hr style='border: 2px solid red;'>

In [4]:
data_raw = pd.read_csv('dn_etl.csv', sep=';', low_memory=False)
data_raw.sort_index(axis=1, inplace=True)

time: 931 ms (started: 2023-09-17 23:21:45 +00:00)


In [5]:
data_raw.shape

(190020, 24)

time: 3.37 ms (started: 2023-09-17 23:21:46 +00:00)


In [6]:
data_raw.head()

,ANO,APGAR1,APGAR5,CODANOMAL_QTD,CONSPRENAT,ESCMAEETL_VALOR,ESTCIVMAE_COMPANHEIRO,GRAVIDEZ_VALOR,LOCNASC_DOMICILIO,LOCNASC_ESTAB_SAUDE,LOCNASC_HOSPITAL,LOCNASC_OUTROS,OBT_NEONATAL,PARTO_CESAREO,PARTO_VAGINALL,PESO,QTDFILMORT,QTDPARTCES,SEMAGESTAC,SEXO_FEM,SEXO_MASC,TPAPRESENT_CEFALICO,TPAPRESENT_PELVICO,TPAPRESENT_TRANSVERSA
0,2015,2.0,4.0,0.0,5.0,1,0,1,0,0,1,0,0,1,0,820.0,1.0,NaN,26.0,0,1,1,0,0
1,2015,9.0,10.0,0.0,8.0,4,1,1,0,0,1,0,0,1,0,4200.0,1.0,NaN,39.0,0,1,1,0,0
2,2015,7.0,8.0,0.0,10.0,11,1,1,0,0,1,0,0,0,1,3700.0,0.0,0.0,40.0,1,0,1,0,0
3,2015,7.0,8.0,0.0,7.0,4,0,1,0,0,1,0,0,0,1,3550.0,1.0,0.0,37.0,0,1,1,0,0
4,2015,9.0,10.0,0.0,3.0,11,0,1,0,0,1,0,0,1,0,3800.0,0.0,NaN,39.0,1,0,1,0,0


time: 33.1 ms (started: 2023-09-17 23:21:47 +00:00)


## Dados X e y<hr style='border: 2px solid red;'>

In [7]:
X_raw= data_raw.drop(columns=['QTDPARTCES', 'ANO', 'OBT_NEONATAL'])
y_raw= data_raw["OBT_NEONATAL"]

time: 17.6 ms (started: 2023-09-17 23:21:51 +00:00)


## Dados desbalanceados<hr style='border: 2px solid red;'>

In [8]:
from imblearn.over_sampling import SMOTE

smt = SMOTE(random_state=SEED)
X_bal, y_bal = smt.fit_resample(X_raw, y_raw)

time: 901 ms (started: 2023-09-17 23:22:03 +00:00)


##Dados de treino e de teste<hr style='border: 2px solid red;'>

In [9]:
X_treino, X_teste, y_treino, y_teste = train_test_split(X_bal, y_bal, test_size =0.3,
                                                        random_state=SEED,
                                                        stratify=y_bal)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(X_treino), len(X_teste)))

Treinaremos com 264068 elementos e testaremos com 113172 elementos
time: 1.12 s (started: 2023-09-17 23:22:06 +00:00)


## Random Forest

In [11]:
modelo_rfc = RandomForestClassifier(n_estimators = 100)
modelo_rfc.fit(X_treino, y_treino)
predito_rfc = modelo_rfc.predict(X_teste)

print("A acurácia do modelo Random Forest foi %.2f%%" % (metrics.accuracy_score(y_teste, predito_rfc) * 100))
print("A Precisão do modelo Random Forest foi %.2f%%" % (metrics.precision_score(y_teste, predito_rfc) * 100))
print("A Recall do modelo Random Forest foi %.2f%%" % (metrics.recall_score(y_teste, predito_rfc) * 100))
print("A F1 do modelo Random Forest foi %.2f%%" % (metrics.f1_score(y_teste, predito_rfc) * 100))

A acurácia do modelo Random Forest foi 99.73%
A Precisão do modelo Random Forest foi 99.82%
A Recall do modelo Random Forest foi 99.63%
A F1 do modelo Random Forest foi 99.73%
time: 45.8 s (started: 2023-09-17 23:22:31 +00:00)


### K-Best <hr style='border: 2px solid red;'>

In [36]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2


time: 8.15 ms (started: 2023-09-18 00:48:37 +00:00)


In [41]:
def rodar_k_best(_modelo):
  count = 1
  df_kbest = pd.DataFrame(columns=['K', 'Acurácia', 'Precisão', 'Recall', 'F1', 'Features'])
  for count in range(1, X_raw.shape[1]):
    selecionar_kmelhores = SelectKBest(chi2, k=count)
    selecionar_kmelhores.fit(X_treino, y_treino)

    mask = selecionar_kmelhores.get_support()
    features = X_treino.columns[mask]
    features_str = reduce(lambda a, b : a+ ", " +str(b), features)

    X_treino_kbest = selecionar_kmelhores.transform(X_treino)
    X_teste_kbest = selecionar_kmelhores.transform(X_teste)

    _modelo.fit(X_treino_kbest, y_treino)
    predito_kb = _modelo.predict(X_teste_kbest)

    metricas = [count,
              round((metrics.accuracy_score(y_teste, predito_kb) * 100), 2),
              round((metrics.precision_score(y_teste, predito_kb) * 100), 2),
              round((metrics.recall_score(y_teste, predito_kb) * 100), 2),
              round((metrics.f1_score(y_teste, predito_kb) * 100), 2),
              features_str]

    df_kbest.loc[count] = metricas
  return df_kbest


time: 5.33 ms (started: 2023-09-18 00:49:46 +00:00)


In [46]:
df_kbest_arvore = rodar_k_best(DecisionTreeClassifier())
df_kbest_arvore

,K,Acurácia,Precisão,Recall,F1,Features
1,1,95.86,98.26,93.36,95.75,PESO
2,2,99.21,99.45,98.97,99.21,"APGAR1, PESO"
3,3,99.41,99.56,99.26,99.41,"APGAR1, APGAR5, PESO"
4,4,99.49,99.49,99.48,99.49,"APGAR1, APGAR5, PESO, SEMAGESTAC"
5,5,99.49,99.45,99.52,99.49,"APGAR1, APGAR5, CONSPRENAT, PESO, SEMAGESTAC"
6,6,99.48,99.46,99.50,99.48,"APGAR1, APGAR5, CODANOMAL_QTD, CONSPRENAT, PES..."
7,7,99.46,99.46,99.47,99.46,"APGAR1, APGAR5, CODANOMAL_QTD, CONSPRENAT, EST..."
8,8,99.48,99.48,99.48,99.48,"APGAR1, APGAR5, CODANOMAL_QTD, CONSPRENAT, EST..."
9,9,99.49,99.43,99.55,99.49,"APGAR1, APGAR5, CODANOMAL_QTD, CONSPRENAT, EST..."
10,10,99.48,99.43,99.53,99.48,"APGAR1, APGAR5, CODANOMAL_QTD, CONSPRENAT, EST..."


time: 1min 10s (started: 2023-09-18 01:21:11 +00:00)


In [45]:
df_kbest_random_forest = rodar_k_best(RandomForestClassifier(n_estimators = 100))
df_kbest_random_forest

,K,Acurácia,Precisão,Recall,F1,Features
1,1,95.86,98.23,93.40,95.76,PESO
2,2,99.19,99.40,98.98,99.19,"APGAR1, PESO"
3,3,99.46,99.57,99.34,99.46,"APGAR1, APGAR5, PESO"
4,4,99.64,99.68,99.60,99.64,"APGAR1, APGAR5, PESO, SEMAGESTAC"
5,5,99.68,99.74,99.63,99.68,"APGAR1, APGAR5, CONSPRENAT, PESO, SEMAGESTAC"
6,6,99.70,99.75,99.65,99.70,"APGAR1, APGAR5, CODANOMAL_QTD, CONSPRENAT, PES..."
7,7,99.71,99.76,99.67,99.71,"APGAR1, APGAR5, CODANOMAL_QTD, CONSPRENAT, EST..."
8,8,99.68,99.72,99.65,99.68,"APGAR1, APGAR5, CODANOMAL_QTD, CONSPRENAT, EST..."
9,9,99.69,99.73,99.66,99.69,"APGAR1, APGAR5, CODANOMAL_QTD, CONSPRENAT, EST..."
10,10,99.69,99.72,99.66,99.69,"APGAR1, APGAR5, CODANOMAL_QTD, CONSPRENAT, EST..."


time: 16min 13s (started: 2023-09-18 00:58:38 +00:00)


### RFE <hr style='border: 2px solid red;'>

In [48]:
from sklearn.feature_selection import RFE

time: 487 µs (started: 2023-09-18 01:22:47 +00:00)


In [62]:
def rodar_rfe(_modelo):
  count = 1
  df_rfe = pd.DataFrame(columns=['K', 'Acurácia', 'Precisão', 'Recall', 'F1', 'Features'])
  # X_raw.shape[1]
  for count in range(1, 5+1):
    selecionador_rfe = RFE(estimator = _modelo, n_features_to_select = count, step = 1)
    selecionador_rfe.fit(X_treino, y_treino)

    mask = selecionador_rfe.get_support()
    features = X_treino.columns[mask]
    features_str = reduce(lambda a, b : a+ ", " +str(b), features)

    X_treino_rfe = selecionador_rfe.transform(X_treino)
    X_teste_rfe = selecionador_rfe.transform(X_teste)

    _modelo.fit(X_treino_rfe, y_treino)
    predito_rfe = _modelo.predict(X_teste_rfe)

    metricas = [count,
              round((metrics.accuracy_score(y_teste, predito_rfe) * 100), 2),
              round((metrics.precision_score(y_teste, predito_rfe) * 100), 2),
              round((metrics.recall_score(y_teste, predito_rfe) * 100), 2),
              round((metrics.f1_score(y_teste, predito_rfe) * 100), 2),
              features_str]

    df_rfe.loc[count] = metricas
  return df_rfe

time: 1.66 ms (started: 2023-09-18 02:58:45 +00:00)


In [59]:
df_rfe_arvore = rodar_rfe(DecisionTreeClassifier())
df_rfe_arvore

,K,Acurácia,Precisão,Recall,F1,Features
1,1,95.60,97.96,93.14,95.49,APGAR5
2,2,99.14,99.50,98.78,99.14,"APGAR5, PESO"
3,3,99.40,99.55,99.25,99.40,"APGAR1, APGAR5, PESO"
4,4,99.48,99.50,99.46,99.48,"APGAR1, APGAR5, PESO, SEMAGESTAC"
5,5,99.51,99.54,99.49,99.51,"APGAR1, APGAR5, CODANOMAL_QTD, PESO, SEMAGESTAC"
6,6,99.54,99.57,99.51,99.54,"APGAR1, APGAR5, CODANOMAL_QTD, PESO, QTDFILMOR..."
7,7,99.53,99.52,99.55,99.53,"APGAR1, APGAR5, CODANOMAL_QTD, PARTO_VAGINALL,..."
8,8,99.54,99.53,99.54,99.54,"APGAR1, APGAR5, CODANOMAL_QTD, PARTO_CESAREO, ..."
9,9,99.54,99.54,99.55,99.54,"APGAR1, APGAR5, CODANOMAL_QTD, ESCMAEETL_VALOR..."


time: 6min 3s (started: 2023-09-18 01:44:38 +00:00)


In [63]:
df_rfe_random_forest = rodar_rfe(RandomForestClassifier(n_estimators = 100))
df_rfe_random_forest

,K,Acurácia,Precisão,Recall,F1,Features
1,1,95.60,97.96,93.14,95.49,APGAR5
2,2,98.17,99.43,96.89,98.14,"APGAR1, APGAR5"
3,3,99.45,99.56,99.33,99.44,"APGAR1, APGAR5, PESO"
4,4,99.64,99.67,99.61,99.64,"APGAR1, APGAR5, PESO, SEMAGESTAC"
5,5,99.69,99.74,99.64,99.69,"APGAR1, APGAR5, CODANOMAL_QTD, PESO, SEMAGESTAC"


time: 1h 13min 8s (started: 2023-09-18 02:58:59 +00:00)


### Cross Validation <hr style='border: 2px solid red;'>

In [ ]:
from sklearn.feature_selection import RFECV

In [ ]:
df_cv = pd.DataFrame(columns=['K', 'Acurácia', 'Precisão', 'Recall', 'F1'])
count = 2

In [ ]:
count = 10

In [ ]:
selecionar_cv = RFECV(estimator = modelo_rfc, cv = count, scoring = "accuracy", step = 1)
selecionar_cv.fit(X_treino, y_treino)

X_treino_cv = selecionar_cv.transform(X_treino)
X_teste_cv = selecionar_cv.transform(X_teste)

modelo_cv = RandomForestClassifier(n_estimators = 100)
modelo_cv.fit(X_treino_cv, y_treino)
predito_cv = modelo_cv.predict(X_teste_cv)

metricas = [count,
            round((metrics.accuracy_score(y_teste, predito_cv) * 100), 2),
            round((metrics.precision_score(y_teste, predito_cv) * 100), 2),
            round((metrics.recall_score(y_teste, predito_cv) * 100), 2),
            round((metrics.f1_score(y_teste, predito_cv) * 100), 2)]

In [ ]:
df_cv.loc[count] = metricas

In [ ]:
df_cv

,K,Acurácia,Precisão,Recall,F1
2,2.0,99.74,99.82,99.67,99.74
3,3.0,99.74,99.82,99.66,99.74


In [ ]:
count = 2
for count in range(2, X_raw.shape[1]):
  selecionar_cv = RFECV(estimator = modelo_rfc, cv = count, scoring = "accuracy", step = 1)
  selecionar_cv.fit(X_treino, y_treino)

  X_treino_cv = selecionar_cv.transform(X_treino)
  X_teste_cv = selecionar_cv.transform(X_teste)

  modelo_cv = RandomForestClassifier(n_estimators = 100)
  modelo_cv.fit(X_treino_cv, y_treino)
  predito_cv = modelo_cv.predict(X_teste_cv)

  metricas = [count,
              round((metrics.accuracy_score(y_teste, predito_cv) * 100), 2),
              round((metrics.precision_score(y_teste, predito_cv) * 100), 2),
              round((metrics.recall_score(y_teste, predito_cv) * 100), 2),
              round((metrics.f1_score(y_teste, predito_cv) * 100), 2)]
  print(count)

  df_cv.loc[count] = metricas

In [ ]:
df_cv

In [ ]:
matriz_confusao = confusion_matrix(teste_y, classificador.predict(teste_rfecv))
sns.heatmap(matriz_confusao, annot = True, fmt = "d").set(xlabel = "Predição", ylabel = "Real")

print("Resultado da classificação %.2f%%" % (classificador.score(teste_rfecv, teste_y)* 100))

In [ ]:
plt.xlabel("Número de exames")
plt.ylabel("Acurácia")

plt.plot(range(1, len(selecionador_rfecv.cv_results_['mean_test_score'] ) + 1), selecionador_rfecv.cv_results_['mean_test_score'])
plt.show